In [1]:
import numpy as np
import csv
import os
import matplotlib.pyplot as plt
from skimage import io
from scipy import misc
from skimage.transform import rescale, resize, downscale_local_mean
import  scipy
import  scipy.cluster.hierarchy as sch
from scipy.cluster.hierarchy import dendrogram
from os import listdir
from os.path import isdir,isfile,join
import matplotlib

from scipy import sparse
from scipy.sparse.linalg import spsolve
from scipy.signal import find_peaks

In [2]:
#return baseline. Method "Asymmetric Least Squares Smoothing" by P. Eilers and H. Boelens in 2005
def baseline_als(y, lam, p, niter=10):
    L = len(y)
    D = sparse.diags([1,-2,1],[0,-1,-2], shape=(L,L-2))
    w = np.ones(L)
    for i in range(niter):
        W = sparse.spdiags(w, 0, L, L)
        Z = W + lam * D.dot(D.transpose())
        z = spsolve(Z, w*y)
        w = p * (y > z) + (1-p) * (y < z)
    return(z)
#return an array divided by the maximum elements in the given array
def normalized(y, max_y = 1):
    z = y/np.amax(y)
    return(z)

In [3]:
folder = '/media/yingtao/08948FA2948F90BC/2021-imaging/41f64428a26fcgc/'

In [4]:
cd $folder

/media/yingtao/08948FA2948F90BC/2021-imaging/41f64428a26fcgc


In [5]:
fluo_data = ['0430-5-exposure150ms-20x-power-5-20-stained-1-2contibousFW/Results_121_a31c_654_boutons_a26f_bouton.csv',
             '0430-5-exposure150ms-20x-power-5-20-stained-1-2contibousFW/Results_122_a31c_654_boutons_a26f_bouton.csv',
             '0430-7-exposure150ms-20x-power-5-20-stained2/Results-a31c-6543-a26f-a6.csv',
             '0430-9-exposure150ms-20x-power-5-20-stained3/Results-a31c-7654-a26f-a6.csv',
             '0430-10-exposure150ms-20x-power-2-20-stained4/Results-a31c-65432-a26f-a6.csv',
             '0430-12-exposure150ms-20x-power-2-20-stained5/Results-a31c-654-a26f-a6.csv',
             '0430-14-exposure150ms-20x-power2-20-stained6/Results-a31c-5432-a26f-a6.csv',
            ]

In [6]:
%matplotlib qt

In [7]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 18}

matplotlib.rc('font', **font)

In [8]:
def plot(fluo_data, segment_n, fig_n):
    segments= []
    with open(fluo_data) as file:
        lines = csv.reader(file)
        for i in lines:
            segments.append(i[1:])
    segments = segments[1:]
    segments = np.array(segments).astype(float)
    #segments = segments[0:1500,:]
    segn = segments.shape[1]
    segments_normed = np.empty([segn,segments.shape[0]])
    for i in range(segn): 
        baseline = baseline_als(segments[:,i], 10e6, 0.001)
        segments_normed[i] = normalized((segments[:,i]-baseline) / baseline)
    peaks_axon, _ = find_peaks(segments_normed[-1], distance=50, height=(0.2,1))
    slice_i = 0
    slice_e = 1500
    fig = plt.figure(fig_n,figsize=(20,10))
    gs = fig.add_gridspec(2, 1, hspace=0, wspace=0)
    (ax1), (ax2) = gs.subplots(sharex='col')
    ax1.plot(segments_normed[-1,:], label='A26f_a6_bouton')
    ax1.plot(peaks_axon, segments_normed[0,peaks_axon], "x")
    ax1.set_xlim(slice_i,slice_e)
    #plt.xlabel('frame')
    #ax1.set_ylabel('Fluo_cali')
    ax1.legend(loc='best')
    for i in range(len(segment_n)):
        ax2.plot(segments_normed[i,], label='A31c_a'+str(segment_n[i])+'_neurites')
    ax2.plot(peaks_axon,[1,] * len(peaks_axon), "x")
    ax2.set_xlim(slice_i,slice_e)
    #plt.xlabel('frame')
    #plt.ylabel('Fluo')
    ax2.legend(loc='upper right')
    plt.show()

In [9]:
num = 0
for data_path, segment_n in zip(fluo_data,[[6,5,4],[6,5,4],[6,5,4,3],[7,6,5,4],[6,5,4,3,2],[6,5,4],[5,4,3,2]]):
    num += 1
    if num == 7:
        plot(data_path,segment_n, num)

findfont: Font family ['normal'] not found. Falling back to DejaVu Sans.
